In [ ]:
!pip install pandas

In [ ]:
import os
import pandas as pd

Load some lookups

In [ ]:
trust_lookup = pd.read_csv('data/trust_ccg_lookup_ods_datapoint.csv').drop(columns=['Name', 'Primary Role Name', 'Geographic Primary Care Organisation Name'])
trust_lookup.columns = ['trust_code', 'CCG21CDH']
ccg = pd.read_csv('data/ref_ccg21.csv')

Load the MHSDS file, reshape (drop average rows, working columns) and attach codes. Save a file with errors in it.

In [ ]:
mhsds = pd.read_csv('source/MHSDS Data_Rstr_Sep_Nov_2020.csv', header=[0,1], na_values=['#DIV/0!']).fillna(0)
mhsds.columns = mhsds.columns.map('_'.join)
mhsds = mhsds.rename(columns={
    'Unnamed: 0_level_0_LEVEL_ONE': 'trust_code',
    'Unnamed: 1_level_0_REPORTING_PERIOD_START': 'reporting_period_start'
})

mhsds = mhsds.loc[~mhsds.trust_code.str.match('.*Average')]
mhsds = mhsds.drop(columns=[
       'Chemical restraint - Injection (Non Rapid Tranquillisation)_BAME',
       'Chemical restraint - Injection (Non Rapid Tranquillisation)_Non-BAME',
       'Chemical restraint - Injection (Non Rapid Tranquillisation)_Unknown/Not Stated',
       'Chemical restraint - Injection (Rapid Tranquillisation)_BAME',
       'Chemical restraint - Injection (Rapid Tranquillisation)_Non-BAME',
       'Chemical restraint - Injection (Rapid Tranquillisation)_Unknown/Not Stated',
       'Chemical restraint - Oral_BAME', 'Chemical restraint - Oral_Non-BAME',
       'Chemical restraint - Oral_Unknown/Not Stated',
       'Chemical restraint - Other (not listed)_BAME',
       'Chemical restraint - Other (not listed)_Non-BAME',
       'Chemical restraint - Other (not listed)_Unknown/Not Stated',
       'Physical restraint - Kneeling_BAME',
       'Physical restraint - Kneeling_Non-BAME',
       'Physical restraint - Kneeling_Unknown/Not Stated',
       'Physical restraint - Other (not listed)_BAME',
       'Physical restraint - Other (not listed)_Non-BAME',
       'Physical restraint - Other (not listed)_Unknown/Not Stated',
       'Physical restraint - Prone_BAME',
       'Physical restraint - Prone_Non-BAME',
       'Physical restraint - Prone_Unknown/Not Stated',
       'Physical restraint - Restrictive escort_BAME',
       'Physical restraint - Restrictive escort_Non-BAME',
       'Physical restraint - Restrictive escort_Unknown/Not Stated',
       'Physical restraint - Seated_BAME',
       'Physical restraint - Seated_Non-BAME',
       'Physical restraint - Seated_Unknown/Not Stated',
       'Physical restraint - Side_BAME', 'Physical restraint - Side_Non-BAME',
       'Physical restraint - Side_Unknown/Not Stated',
       'Physical restraint - Standing_BAME',
       'Physical restraint - Standing_Non-BAME',
       'Physical restraint - Standing_Unknown/Not Stated',
       'Physical restraint - Supine_BAME',
       'Physical restraint - Supine_Non-BAME',
       'Physical restraint - Supine_Unknown/Not Stated'
])
mhsds = mhsds.merge(trust_lookup, how='left', on='trust_code').merge(ccg, how='left', on='CCG21CDH')
mhsds.loc[mhsds.CCG21CD.isnull()].to_csv('scratch/mhsds_errors.csv')
mhsds = mhsds.loc[~mhsds.CCG21CD.isnull()]

Create the output file, aggegating by average on the CCG21 codes, and tidy up column names.

In [ ]:
output = mhsds.groupby('CCG21CD').mean().round(3)
output.columns = output.columns.str.replace(r'(.*)_(.*)', r'\1 (\2)', regex=True)

Save to csv file

In [ ]:
output.to_csv('data/mhsds.processed.csv')